### Submission Information:
- Group: 04
- Course: BIG DATA AND ITS APPLICATIONS
- Instructor: TS. Nguyễn Thôn Dã
- Assignment: PySpark Practice #5

| StudentID      | Name | Class     |
| :---        |    :---:   |          :--- |
| K204110855 | Đinh Văn An | K20411 |
| K204110568 | Nguyễn Gia Hưng | K20411 |
| K204110579 | Phạm Nguyễn Hiền Phương | K20411 |
| K204111772 | Đinh Hoàn Hảo | K20411 |
| K204110567 | Phạm Thị Minh Hòa | K20411 |
| K194081062 | Lê Thanh Hải | K19408T |

## Import Libraries

In [78]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime

## 4-1-1. Load Data

### Credit card frad Dataset description

| Field      | Description | Data Type |
| :---       |  :---       | :---      |
|transdatetrans_time|	Transaction DateTime | DateTime|
|merchant|	Merchant Name |    String|
|category|	Category of Merchant | String|
|amt|	Amount of Transaction | Float|
|city|	City of Credit Card Holder | String|
|state|	State of Credit Card Holder | String|
|lat|	Latitude Location of Purchase | Float|
|long|	Longitude Location of Purchase | Float|
|city_pop|	Credit Card Holder's City Population | Integer|
|job|	Job of Credit Card Holder | String|
|dob|	Date of Birth of Credit Card Holder | DateTime|
|trans_num|	Transaction Number | String|
|merch_lat|	Latitude Location of Merchant | Float|
|merch_long|	Longitude Location of Merchant | Float|
|is_fraud|	Whether Transaction is Fraud (1) or Not (0) | Integer|

In [35]:
spark = SparkSession.builder.appName('PySpark').getOrCreate()
file_location="Data\credit-card-fraud.csv"
file_type="csv"
infer_schema="False" 
first_row_is_header="True"
df_Origin = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).load(file_location)
print(df_Origin.dtypes)

#save dataframe to csv file
#df_Origin.toPandas().to_csv('Data\monitor.csv', index=False)

[('trans_date_trans_time', 'string'), ('merchant', 'string'), ('category', 'string'), ('amt', 'string'), ('city', 'string'), ('state', 'string'), ('lat', 'string'), ('long', 'string'), ('city_pop', 'string'), ('job', 'string'), ('dob', 'string'), ('trans_num', 'string'), ('merch_lat', 'string'), ('merch_long', 'string'), ('is_fraud', 'string')]


#### Casting

In [45]:
#casting

df = df_Origin
#identifying and assigning the correct data type to each column
float_vars = ['amt', 'lat', 'long', 'merch_lat', 'merch_long']
date_vars = ['dob']
datetime_vars = ['trans_date_trans_time']
int_vars = ['city_pop', 'is_fraud']
for var in float_vars:
    df = df.withColumn(var, df[var].cast(DoubleType()))
for var in date_vars:
    df = df.withColumn(var, df[var].cast(DateType()))
for var in datetime_vars:
    df = df.withColumn(var, df[var].cast(TimestampType()))
for var in int_vars:
    df = df.withColumn(var, df[var].cast(IntegerType()))
#save dataframe to csv file
df.toPandas().to_csv('Data\monitor.csv', index=False)

In [46]:
df.printSchema()
print(df.dtypes)
df.count()

root
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)

[('trans_date_trans_time', 'timestamp'), ('merchant', 'string'), ('category', 'string'), ('amt', 'double'), ('city', 'string'), ('state', 'string'), ('lat', 'double'), ('long', 'double'), ('city_pop', 'int'), ('job', 'string'), ('dob', 'date'), ('trans_num', 'string'), ('merch_lat', 'double'), ('merch_long', 'double'), ('is_fraud', 'int')]


1333

In [47]:
df_Origin.show(3)
df.show(3)

+---------------------+--------------------+-------------+------+----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|     category|   amt|      city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+-------------+------+----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 00:00:44|"""Heller Gutmann...|  grocery_pos|107.23|    Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac...|43.150704|-112.154481|       0|
|  2019-01-01 00:07:27|       

## 4-1-2. Calculating The Age Of Credit Card Holder and Adding It as a Column

In [84]:
df_Age = df.withColumn('Age', round(datediff(lit(current_date()), df['dob'])/lit(365), 0))
df_Age.show(3)

+---------------------+--------------------+-------------+------+----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+----+
|trans_date_trans_time|            merchant|     category|   amt|      city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud| Age|
+---------------------+--------------------+-------------+------+----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+----+
|  2019-01-01 00:00:44|"""Heller Gutmann...|  grocery_pos|107.23|    Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0|44.0|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac...|43.150704|-112.154481|       0|61.0|
|  20